<a href="https://colab.research.google.com/github/tsparaskevas/ML_EDDE2/blob/main/Final%20essay/greek_news_sites_scrappers_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Import libraries**

In [ ]:
from datetime import date 
from datetime import datetime
import sys
import os
from pathlib import Path
import pandas as pd

import re

import requests
from bs4 import BeautifulSoup
import time

from google.colab import drive 
drive.mount('gdrive') 

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


####**Define Search pages and Keywords**

Define start and last page numbers

In [ ]:
# Start page number
#s = 85 #85
# Last page number
#l = 87 #135

Define search term

In [ ]:
#search_term = "τροχαίο"

###**Newsbomb.gr**

**Search pages function**

In [ ]:
def scrape_newsbomb_search_pages(s,l):
  base_url = "https://www.newsbomb.gr/search?q=" # το βασικό url των σελίδων του search
  main_url = base_url + search_term + "&page="
  global teaser_soups_list
  teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

  # Scraping των ζητούμενων σελίδων Latest
  for i in range(s, l): 
    page_url = main_url + str(i) 
    page = requests.get(page_url) 
    teaser_soup = BeautifulSoup(page.text, 'html.parser') 
    print(f"Now Scraping page: {i}") 
    teaser_soups_list.append(teaser_soup) 
    time.sleep(2)

  # Συλλογή των δεδομένων των teaser articles
  global teaser_articles_list
  teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
  nbr = 0
  for teaser_soup in teaser_soups_list:
    teaser_articles = teaser_soup.find('div', {'class': 'list-items viewbox'}).find_all('div', {'class': 'ctype-story'})
    for teaser_article in teaser_articles:
      teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
      a = teaser_article.find('a', {'class': 'item-link'})
      if a:
        teaser_article_dict['Url'] = "https://www.newsbomb.gr" + a['href'] # url
      h3 = teaser_article.find('h3')
      if h3:
        teaser_article_dict['Title'] = h3.text.replace('\n', "").replace('\t', "") # title
      lead = teaser_article.find('div', {'class': 'item-intro'})
      if lead:
        teaser_article_dict['Lead'] = lead.text.replace('\n', "").replace('\t', "") # lead
      category = teaser_article.find('span', {'class': 'item-category'})
      if category:
        teaser_article_dict['Section'] = category.text # category
      teaser_articles_list.append(teaser_article_dict)
  # Create dataframe
  #teasers_df = pd.DataFrame(teaser_articles_list)
  #print(f"Teasers' dataframe is created. It contains: {len(teaser_articles_list)}")

**Full articles function**

In [ ]:
def scrape_newsbomb_full_articles():
  # Scraping όλων των σελίδων των ολόκληρων άρθρων
  full_soups_list = [] 
  nbr = 0 
  global teasers_df
  for article_url in teasers_df['Url']: 
    nbr += 1 
    full_response = requests.get(article_url) 
    full_soup = BeautifulSoup(full_response.text, 'html.parser')
    full_soups_list.append(full_soup)
    time.sleep(2)
    print(f'Scraping article {nbr} of {len(teasers_df)}')

  # Συλλογή των δεδομένων για τα ολόκληρα άρθρα
  global full_articles_list
  full_articles_list = []
  nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του teasers_df, να φέρνω και το αντίστοιχο url του άρθρου
  for soup in full_soups_list:
    print(f"working on article {teasers_df.iloc[[int(nbr)]]['Url'].item()}")
    full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
    full_article_dict['Title'] = teasers_df.iloc[[int(nbr)]]['Title'].item() # Title (από το teasers_df)
    full_article_dict['Url'] = teasers_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου (από το teasers_df)
    full_article_dict['Lead'] = teasers_df.iloc[[int(nbr)]]['Lead'].item() # το Lead του άρθρου (από το teasers_df)
    full_article_dict['Section'] = teasers_df.iloc[[int(nbr)]]['Section'].item() # το Url του άρθρου (από το teasers_df)
    try:
      dtime = soup.find('time') # Time
      if dtime:
        datetime_str = dtime.text.replace('\n', '').replace('\t', '') + ":00"
        full_article_dict['Date'] = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M:%S')
    except:
      pass
    try:
      author = soup.find('span', {'class': 'author-name'}) # Author Newsroom
      if author:
        full_article_dict['Author'] = author.text
    except:
      pass
    try:
      author = soup.find('a', {'class': 'author-name'}) # Author
      if author:
        full_article_dict['Author'] = author.text
    except:
      pass
    try:
      paragraphsL = soup.find('div', {'class': 'main-text story-fulltext'}).find_all('p')
      if paragraphsL:
        paragraphsL = paragraphsL[:-2]
        ptextsL = []
        for p in paragraphsL: # Body
          p_text = p.text
          ptextsL.append(p_text)
        body = (' ').join(ptextsL)
        full_article_dict['Text'] = body
    except:
      pass
    full_articles_list.append(full_article_dict)
    nbr += 1
  # Create dataframe
  #full_articles_df = pd.DataFrame(full_articles_list)
  #print(f"Full articles' dataframe is created. It contains: {len(full_articles_list)}")

###**Dikaiologitika.gr**

**Search pages function**

In [ ]:
def scrape_dikaiologitika_search_pages(s,l):
  base_url = "https://www.dikaiologitika.gr/site/search/" # το βασικό url των σελίδων του search
  main_url = base_url + search_term + "?searchword=" + search_term + "&start="
  global teaser_soups_list
  teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

  # Scraping των ζητούμενων σελίδων Latest
  for i in range(s, l+1, 20): 
    page_url = main_url + str(i) 
    page = requests.get(page_url) 
    teaser_soup = BeautifulSoup(page.text, 'html.parser') 
    print(f"Now Scraping page: {i}") 
    teaser_soups_list.append(teaser_soup) 
    time.sleep(2)

  # Συλλογή των δεδομένων των teaser articles
  global teaser_articles_list
  teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
  nbr = 0
    
  # Scraping των ζητούμενων σελίδων Latest
  for teaser_soup in teaser_soups_list:
    teaser_articles = teaser_soup.find('div', {'class': 'itemListLowerGrid'}).find_all('div', {'class': 'itemContainer'})
    for teaser_article in teaser_articles:
      teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
      h3_a = teaser_article.find('h3', {'class': 'genericItemTitle'}).find('a')
      if h3_a:
        teaser_article_dict['Url'] = "https://www.dikaiologitika.gr" + h3_a['href'] # url
        teaser_article_dict['Title'] = h3_a.text # title
      else:
        teaser_article_dict['Url'] = " "
        teaser_article_dict['Title'] = " "       
      category = teaser_article.find('span', {'class': 'genericItemCategory'})
      if category:
        teaser_article_dict['Section'] = category.text # section
      else:
        teaser_article_dict['Section'] = " "
      dtime = teaser_article.find('span', {'class', 'genericItemDateCreated'}) 
      if dtime:
        datetime_str = dtime.text.replace('- ', '') + ":00"
        teaser_article_dict['Date'] = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M:%S') # date
      else:
        teaser_article_dict['Date'] = " "
      teaser_articles_list.append(teaser_article_dict)
  print(f'# of dictionaries created: {len(teaser_articles_list)}')

  # Create dataframe
  #teasers_df = pd.DataFrame(teaser_articles_list)
  #print(f"Teasers' dataframe is created. It contains: {len(teaser_articles_list)}")

**Full articles function**

In [ ]:
def scrape_dikaiologitika_full_articles():
  # Scraping όλων των σελίδων των ολόκληρων άρθρων
  full_soups_list = [] 
  nbr = 0 
  global teasers_df
  for article_url in teasers_df['Url']: 
    nbr += 1 
    full_response = requests.get(article_url) 
    full_soup = BeautifulSoup(full_response.text, 'html.parser')
    full_soups_list.append(full_soup)
    time.sleep(2)
    print(f'Scraping article {nbr} of {len(teasers_df)}')

  # Συλλογή των δεδομένων για τα ολόκληρα άρθρα
  global full_articles_list
  full_articles_list = []
  nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του teasers_df, να φέρνω και το αντίστοιχο url του άρθρου
  for soup in full_soups_list:
    print(f"working on article {teasers_df.iloc[[int(nbr)]]['Url'].item()}")
    full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
    full_article_dict['Title'] = teasers_df.iloc[[int(nbr)]]['Title'].item() # Title (από το teasers_df)
    full_article_dict['Url'] = teasers_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου (από το teasers_df)
    try:
      lead = soup.find('div', {'class': 'itemIntroText'}) 
      if lead:
        full_article_dict['Lead'] = lead.text.replace('\n', '').replace('  ', '') # Lead
    except:
      pass
    full_article_dict['Section'] = teasers_df.iloc[[int(nbr)]]['Section'].item() # το Section του άρθρου (από το teasers_df)
    full_article_dict['Date'] = teasers_df.iloc[[int(nbr)]]['Date'].item() # το Date του άρθρου (από το teasers_df)
    try:
      author = soup.find('div', {'class': 'itemAuthor'}).find('a') 
      if author:
        full_article_dict['Author'] = author.text[1:] # Author Newsroom
    except:
      pass
    try:
      paragraphsL = soup.find('div', {'class': 'itemFullText'}).find_all('p')
      if paragraphsL:
        ptextsL = []
        for p in paragraphsL: # Body
          p_text = p.text.replace('\xa0', ' ').replace('\n', '').replace('  ', '')
          ptextsL.append(p_text)
        body = (' ').join(ptextsL)
        full_article_dict['Text'] = body
      else:
        full_article_dict['Text'] = " "        
    except:
      pass
    full_articles_list.append(full_article_dict)
    nbr += 1
  
  # Create dataframe
  #full_articles_df = pd.DataFrame(full_articles_list)
  #print(f"Full articles' dataframe is created. It contains: {len(full_articles_list)}")

###**Newsit.gr**

**Search pages function**

In [ ]:
def scrape_newsit_search_pages(s,l):
  base_url = "https://www.newsit.gr/page/" # το βασικό url των σελίδων του search
  global teaser_soups_list
  teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

  # Scraping των ζητούμενων σελίδων Latest
  for i in range(s, l):
    page_url = base_url + str(i) + "/?s=" + search_term 
    page = requests.get(page_url) 
    teaser_soup = BeautifulSoup(page.text, 'html.parser') 
    print(f"Now Scraping page: {i}") 
    teaser_soups_list.append(teaser_soup) 
    time.sleep(2)

  # Συλλογή των δεδομένων των teaser articles
  global teaser_articles_list
  teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
  nbr = 0
  for teaser_soup in teaser_soups_list:
    teaser_articles = teaser_soup.find('main', {'class': 'site-main'}).find_all('article')
    for teaser_article in teaser_articles:
      teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
      h2_a = teaser_article.find('h2').find('a')
      if h2_a:
        teaser_article_dict['Url'] = h2_a['href'] # url
        teaser_article_dict['Title'] = h2_a.text # title
      dtime = teaser_article.find('time', {'class': 'entry-date published'})
      if dtime:
        teaser_article_dict['Date'] = dtime['datetime'].replace('T', ' ')[:-6] # date
      teaser_articles_list.append(teaser_article_dict)
  print(f'# of dictionaries created: {len(teaser_articles_list)}')
  
  # Create dataframe
  #teasers_df = pd.DataFrame(teaser_articles_list)
  #print(f"Teasers' dataframe is created. It contains: {len(teaser_articles_list)}")

**Full articles function**

In [ ]:
def scrape_newsit_full_articles():
  # Scraping όλων των σελίδων των ολόκληρων άρθρων
  full_soups_list = [] 
  nbr = 0 
  global teasers_df
  for article_url in teasers_df['Url']: 
    nbr += 1 
    full_response = requests.get(article_url) 
    full_soup = BeautifulSoup(full_response.text, 'html.parser')
    full_soups_list.append(full_soup)
    time.sleep(2)
    print(f'Scraping article {nbr} of {len(teasers_df)}')

  # Συλλογή των δεδομένων για τα ολόκληρα άρθρα
  global full_articles_list
  full_articles_list = []
  nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του teasers_df, να φέρνω και το αντίστοιχο url του άρθρου
  for soup in full_soups_list:
    print(f"working on article {teasers_df.iloc[[int(nbr)]]['Url'].item()}")
    full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
    full_article_dict['Title'] = teasers_df.iloc[[int(nbr)]]['Title'].item() # Title (από το teasers_df)
    full_article_dict['Url'] = teasers_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου (από το teasers_df)
    try:
      lead = soup.find('div', {'class': 'itemIntroText'}) # Lead
      if lead:
        full_article_dict['Lead'] = lead.text.replace('\n', '').replace('  ', '')
      else:
        full_article_dict['Lead'] = " "
    except:
      pass
    try:
      section = soup.find('p', {'id': 'breadcrumbs'})
      if section:
        full_article_dict['Section'] = section.find_all('a')[1].text # Section
    except:
      pass
    full_article_dict['Date'] = teasers_df.iloc[[int(nbr)]]['Date'].item() # το Date του άρθρου (από το teasers_df)
    try:
      author = soup.find('a', {'rel': 'author'}) # Author
      if author:
        full_article_dict['Author'] = author.text
      else:
        full_article_dict['Author'] = " "
    except:
      pass
    try:
      content = soup.find('div', {'class': 'entry-content post-with-no-excerpt'})
      if content:
        paragraphsL = content.find_all('p')
        ptextsL = []
        for p in paragraphsL: # Body
          p_text = p.text.replace('\n', '').replace('\xa0', '').replace('ΔΙΑΦΗΜΙΣΗ', '').replace('  ', '')
          ptextsL.append(p_text)
        body = (' ').join(ptextsL)
        full_article_dict['Text'] = body
      else:
        content = soup.find('div', {'class': 'inside-article'})
        if content:
          paragraphsL = content.find_all('p')
          ptextsL = []
          for p in paragraphsL:
            p_text = p.text.replace('…', '').replace("ΔΙΑΦΗΜΙΣΗ", '').replace('ΟΛΕΣ ΟΙ ΕΙΔΗΣΕΙΣ', '')
            ptextsL.append(p_text)
          body = (' ').join(ptextsL)
          full_article_dict['Text'] = body
######
#    try:
#      paragraphsL = soup.find('div', {'class': 'entry-content post-with-no-excerpt'}).find_all('p')
#      if paragraphsL:
#        ptextsL = []
#        for p in paragraphsL: # Body
#          p_text = p.text.replace('\n', '').replace('\xa0', '').replace('ΔΙΑΦΗΜΙΣΗ', '').replace('  ', '')
#          ptextsL.append(p_text)
#        body = (' ').join(ptextsL)
#        full_article_dict['Text'] = body
#####
    except:
      pass
    full_articles_list.append(full_article_dict)
    nbr += 1
  
  # Create dataframe
  #full_articles_df = pd.DataFrame(full_articles_list)
  #print(f"Full articles' dataframe is created. It contains: {len(full_articles_list)}")

###**Cnn.gr**

**Search pages function**

In [ ]:
def scrape_cnn_search_pages(s,l):
  base_url = "https://www.cnn.gr/search?q=" # το βασικό url των σελίδων του search
  main_url = base_url + search_term + "&page="
  global teaser_soups_list
  teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

  # Scraping των ζητούμενων σελίδων Latest
  for i in range(s, l):
    page_url = main_url + str(i) 
    page = requests.get(page_url) 
    teaser_soup = BeautifulSoup(page.text, 'html.parser') 
    print(f"Now Scraping page: {i}") 
    teaser_soups_list.append(teaser_soup) 
    time.sleep(2)

  # Συλλογή των δεδομένων των teaser articles
  global teaser_articles_list
  teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
  nbr = 0
  for teaser_soup in teaser_soups_list:
    teaser_articles = teaser_soup.find('div', {'class': 'list-items flat-cards'}).find_all('div', {'class': 'gtr'})
    for teaser_article in teaser_articles:
      teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
      a = teaser_article.find('a')
      if a:
        teaser_article_dict['Url'] = "https://www.cnn.gr" + a['href'] # url
      h3 = teaser_article.find('h3')
      if h3:
        teaser_article_dict['Title'] = h3.text.replace('\n', "").replace(' \t', "").replace('\t', '') # title
      teaser_articles_list.append(teaser_article_dict)
  print(f'# of dictionaries created: {len(teaser_articles_list)}')

  # Create dataframe
  #teasers_df = pd.DataFrame(teaser_articles_list)
  #print(f"Teasers' dataframe is created. It contains: {len(teaser_articles_list)}")

**Full articles function**

In [ ]:
def scrape_cnn_full_articles():
  # Scraping όλων των σελίδων των ολόκληρων άρθρων
  full_soups_list = [] 
  nbr = 0 
  global teasers_df
  for article_url in teasers_df['Url']: 
    nbr += 1 
    full_response = requests.get(article_url) 
    full_soup = BeautifulSoup(full_response.text, 'html.parser')
    full_soups_list.append(full_soup)
    time.sleep(2)
    print(f'Scraping article {nbr} of {len(teasers_df)}')

  # Συλλογή των δεδομένων για τα ολόκληρα άρθρα
  global full_articles_list
  full_articles_list = []
  nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του teasers_df, να φέρνω και το αντίστοιχο url του άρθρου
  for soup in full_soups_list:
    print(f"working on article {teasers_df.iloc[[int(nbr)]]['Url'].item()}")
    full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
    full_article_dict['Title'] = teasers_df.iloc[[int(nbr)]]['Title'].item() # Title (από το teasers_df)
    full_article_dict['Url'] = teasers_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου (από το teasers_df)
    try:
      lead = soup.find('div', {'class': 'main-intro story-intro'}) # Lead
      if lead:
        full_article_dict['Lead'] = lead.find('p').text
      else:
        full_article_dict['Lead'] = " "
    except:
      pass
    try:
      section = soup.find('a', {'class': 'main-item-category'})
      if section:
        full_article_dict['Section'] = section.text.replace('\n', '').replace('\t', '') # Section
    except:
      pass
    try:
      dtime = soup.find('time')
      if datetime:
        full_article_dict['Date'] = dtime['datetime'].replace('T', ' ')[:-6] # Date
    except:
      pass
    try:
      author = soup.find('span', {'class': 'author-name'}) # Author Newsroom
      if author:
        full_article_dict['Author'] = author.text
      else:
        full_article_dict['Author'] = " "
    except:
      pass
    try:
      paragraphsL = soup.find('div', {'class': 'main-text story-fulltext'}).find_all('p')
      if paragraphsL:
        ptextsL = []
        for p in paragraphsL: # Body
          p_text = p.text
          ptextsL.append(p_text)
        body = (' ').join(ptextsL)
        full_article_dict['Text'] = body
    except:
      pass
    full_articles_list.append(full_article_dict)
    nbr += 1

  # Create dataframe
  #full_articles_df = pd.DataFrame(full_articles_list)
  #print(f"Full articles' dataframe is created. It contains: {len(full_articles_list)}")

###**Lifo.gr**

**Search pages function**

In [ ]:
def scrape_lifo_search_pages(s,l):
  base_url = "https://www.lifo.gr/search?keyword=" # το βασικό url των σελίδων του search
  main_url = base_url + search_term + "&sort_by=field_publication_date&_wrapper_format=html&page="
  global teaser_soups_list
  teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

  # Scraping των ζητούμενων σελίδων Latest
  for i in range(s, l):
    page_url = main_url + str(i)
    page = requests.get(page_url) 
    teaser_soup = BeautifulSoup(page.text, 'html.parser') 
    print(f"Now Scraping page: {i}") 
    teaser_soups_list.append(teaser_soup) 
    time.sleep(2)

  # Συλλογή των δεδομένων των teaser articles
  global teaser_articles_list
  teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
  nbr = 0
  for teaser_soup in teaser_soups_list:
    teaser_articles = teaser_soup.find('div', {'class': 'container p-0'}).find_all('article')
    for teaser_article in teaser_articles:
      teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
      h3 = teaser_article.find('h3').find_all('a')[1]
      if h3:
        teaser_article_dict['Url'] = h3['href'] # url
        teaser_article_dict['Title'] = h3.text[1:-1] # title
      dtime = teaser_article.find('time')
      if dtime:
        teaser_article_dict['Date'] = dtime['datetime'].replace('T', ' ')[:-5] # date
      lead = teaser_article.find('span', {'class': 'm-0 p-0 fedrabook fs-6-v lh-10-v font-italic d-none d-lg-block darkText'})
      if lead:
        teaser_article_dict['Lead'] = lead.text[1:-1] # lead
      section = teaser_article.find('h3').find('span', {'class': ' grayText slash'})
      if section:
        teaser_article_dict['Section'] = section.text # section
      teaser_articles_list.append(teaser_article_dict)
  print(f'# of dictionaries created: {len(teaser_articles_list)}')

  # Create dataframe
  #teasers_df = pd.DataFrame(teaser_articles_list)
  #print(f"Teasers' dataframe is created. It contains: {len(teaser_articles_list)}")

**Full articles function**

In [ ]:
def scrape_lifo_full_articles():
  # Scraping όλων των σελίδων των ολόκληρων άρθρων
  full_soups_list = [] 
  nbr = 0 
  global teasers_df
  for article_url in teasers_df['Url']: 
    nbr += 1 
    full_response = requests.get(article_url) 
    full_soup = BeautifulSoup(full_response.text, 'html.parser')
    full_soups_list.append(full_soup)
    time.sleep(2)
    print(f'Scraping article {nbr} of {len(teasers_df)}')

  # Συλλογή των δεδομένων για τα ολόκληρα άρθρα
  global full_articles_list
  full_articles_list = []
  nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του teasers_df, να φέρνω και το αντίστοιχο url του άρθρου
  for soup in full_soups_list:
    print(f"working on article {teasers_df.iloc[[int(nbr)]]['Url'].item()}")
    full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
    full_article_dict['Title'] = teasers_df.iloc[[int(nbr)]]['Title'].item() # Title (από το teasers_df)
    full_article_dict['Url'] = teasers_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου (από το teasers_df)
    full_article_dict['Lead'] = teasers_df.iloc[[int(nbr)]]['Lead'].item() # το Lead του άρθρου (από το teasers_df)
    full_article_dict['Section'] = teasers_df.iloc[[int(nbr)]]['Section'].item() # το Section του άρθρου (από το teasers_df)
    full_article_dict['Date'] = teasers_df.iloc[[int(nbr)]]['Date'].item() # το Date του άρθρου (από το teasers_df)
    try:
      author = soup.find('a', {'class': 'article__author'}).find('span') # Author Newsroom
      if author:
        full_article_dict['Author'] = author.text
      else:
        full_article_dict['Author'] = " "
    except:
      pass
    try:  
      ptL = []
      textL = soup.find('div', {'class': 'article__body article__body--mcap'}).find_all('p')   #bodycontent episode-long-desc
      if textL:
        for p in textL:
            if p.find(class_='article__media article__media--caption'):
                continue
            else:
              p = p.text.replace('\xa0', ' ').replace('  ', '')
              ptL.append(p)
        body = (' ').join(ptL)
        full_article_dict['Text'] = body
      else:
        textL = soup.find('div', {'class': 'episode-long-desc'}).find_all('p')
        if textL:
          for p in textL:
              if p.find(class_='article__media article__media--caption'):
                  continue
              else:
                p = p.text.replace('\xa0', ' ').replace('  ', '')
                ptL.append(p)
          body = (' ').join(ptL)
          full_article_dict['Text'] = body
        else:
          full_article_dict['Text'] = " "
    except:
      pass    
    full_articles_list.append(full_article_dict)
    nbr += 1
  # Create dataframe
  #full_articles_df = pd.DataFrame(full_articles_list)
  #print(f"Full articles' dataframe is created. It contains: {len(full_articles_list)}")

###**Naftemporiki.gr**

**Define search pages, keywords AND start/end date**

In [ ]:
#search_term = "τροχαίο"

In [ ]:
#if ' ' in search_term:
#  termsL = search_term.split()
#  search_term = '+'.join(termsL)
#else:
#  search_term = search_term
#print(search_term)

In [ ]:
#start_date = "2020-01-01"
#end_date = "2020-12-31"

In [ ]:
#s = 1
#l = 2

**Search pages function**

In [ ]:
def scrape_naftemporiki_search_pages(s,l):
  base_url = "https://www.naftemporiki.gr/search?q=" # το βασικό url των σελίδων του search
  #main_url = base_url + search_term + "&sort_by=field_publication_date&_wrapper_format=html&page="
  global teaser_soups_list
  teaser_soups_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των html των σελίδων των κατηγοριών

  # Scraping των ζητούμενων σελίδων Latest
  for i in range(s, l):
    page_url = base_url + search_term + "&FQ=pubdate%3a%5b" + start_date + "T00%3a00%3a00%2b02%3a00Z+TO+" + end_date + "T00%3a00%3a00%2b02%3a00Z%5d&sort=pubdate+desc&page=" + str(i)
  #  page_url = main_url + str(i)
    page = requests.get(page_url) 
    teaser_soup = BeautifulSoup(page.text, 'html.parser') 
    print(f"Now Scraping page: {i}") 
    teaser_soups_list.append(teaser_soup) 
    time.sleep(2)

  # Συλλογή των δεδομένων των teaser articles
  global teaser_articles_list
  teaser_articles_list = [] # δημιουργία κενής λίστας για την αποθήκευση όλων των λεξικών με τα δεδομένα κάθε teaser άρθρου
  nbr = 0
  for teaser_soup in teaser_soups_list:
    teaser_articles = teaser_soup.find_all('div', {'class': 'summary'})
    for teaser_article in teaser_articles:
      teaser_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε άρθρου
      h4_a = teaser_article.find('h4').find('a')
      if h4_a:
        teaser_article_dict['Url'] = "https://naftemporiki.gr" + h4_a['href'] # url
        #print("https://naftemporiki.gr" + h4_a['href'])
        teaser_article_dict['Title'] = h4_a.text.replace('\r', '').replace('\n', '').replace('  ', '') # title
      datetime_obj = teaser_article.find('span', {'class': 'topicDate'}) # datetime
      if datetime_obj:
        datetime_str = datetime_obj.text.replace('\r', '').replace('\n', '').replace(',', '').replace('- ', '').replace('  ', '') + ":00"
        datetime_str = re.sub('Δευτέρα |Τρίτη |Τετάρτη |Πέμπτη |Παρασκευή |Σάββατο |Κυριακή ', '', datetime_str)
        datetime_str = re.sub('Ιανουαρίου', '01', datetime_str)
        datetime_str = re.sub('Φεβρουαρίου', '02', datetime_str)
        datetime_str = re.sub('Μαρτίου', '03', datetime_str)
        datetime_str = re.sub('Απριλίου', '04', datetime_str)
        datetime_str = re.sub('Μαΐου', '05', datetime_str)
        datetime_str = re.sub('Ιουνίου', '06', datetime_str)
        datetime_str = re.sub('Ιουλίου', '07', datetime_str)
        datetime_str = re.sub('Αυγούστου', '08', datetime_str)
        datetime_str = re.sub('Σεπτεμβρίου', '09', datetime_str)
        datetime_str = re.sub('Οκτωβρίου', '10', datetime_str)
        datetime_str = re.sub('Νοεμβρίου', '11', datetime_str)
        datetime_str = re.sub('Δεκεμβρίου', '12', datetime_str)
        teaser_article_dict['Date'] = datetime.strptime(datetime_str, '%d %m %Y %H:%M:%S') # date
      teaser_article_dict['Lead'] = ' ' # No lead
      section = teaser_article.find('div', {'class': 'right'})
      if section:
        teaser_article_dict['Section'] = section.text.replace('\r', '').replace('\n', '').replace('  ', '') # section
      teaser_articles_list.append(teaser_article_dict)
  print(f'# of dictionaries created: {len(teaser_articles_list)}')

  # Create dataframe
  #teasers_df = pd.DataFrame(teaser_articles_list)
  #print(f"Teasers' dataframe is created. It contains: {len(teaser_articles_list)}")

**Full articles function**

In [ ]:
def scrape_naftemporiki_full_articles():
  # Scraping όλων των σελίδων των ολόκληρων άρθρων
  full_soups_list = [] 
  nbr = 0 
  global teasers_df
  for article_url in teasers_df['Url']: 
    nbr += 1 
    full_response = requests.get(article_url) 
    full_soup = BeautifulSoup(full_response.text, 'html.parser')
    full_soups_list.append(full_soup)
    time.sleep(2)
    print(f'Scraping article {nbr} of {len(teasers_df)}')

  # Συλλογή των δεδομένων για τα ολόκληρα άρθρα
  global full_articles_list
  full_articles_list = []
  nbr = 0 # Δημιουργώ αρίθμηση ώστε παραπέμποντας στο index του teasers_df, να φέρνω και το αντίστοιχο url του άρθρου
  for soup in full_soups_list:
    print(f"working on article {teasers_df.iloc[[int(nbr)]]['Url'].item()}")
    full_article_dict = {} # δημιουργία κενού λεξικού για την αποθήκευση των δεδομένων κάθε ιστορίας-άρθρου
    full_article_dict['Title'] = teasers_df.iloc[[int(nbr)]]['Title'].item() # Title (από το teasers_df)
    full_article_dict['Url'] = teasers_df.iloc[[int(nbr)]]['Url'].item() # το Url του άρθρου (από το teasers_df)
    full_article_dict['Lead'] = teasers_df.iloc[[int(nbr)]]['Lead'].item() # το Lead του άρθρου (από το teasers_df)
    full_article_dict['Section'] = teasers_df.iloc[[int(nbr)]]['Section'].item() # το Section του άρθρου (από το teasers_df)
    full_article_dict['Date'] = teasers_df.iloc[[int(nbr)]]['Date'].item() # το Date του άρθρου (από το teasers_df)
    author = soup.find('a', {'class': 'article__author'})#.find('span') # Author Newsroom
    if author:
      full_article_dict['Author'] = author.find('span').text
    else:
      full_article_dict['Author'] = " "
    try:
      paragraphsL = soup.find('span', {'id': 'spBody'}).find_all('p')
      if paragraphsL:
        ptextsL = []
        for p in paragraphsL: # Body
          p_text = p.text.replace('\xa0', ' ').replace('\n', '').replace('  ', ' ')
          ptextsL.append(p_text)
        body = (' ').join(ptextsL)
        full_article_dict['Text'] = body
    except:
      pass
    full_articles_list.append(full_article_dict)
    nbr += 1

  # Create dataframe
  #full_articles_df = pd.DataFrame(full_articles_list)
  #print(f"Full articles' dataframe is created. It contains: {len(full_articles_list)}")